# **Polyhedra**
$\DeclareMathOperator*{\conv}{conv}$
$\DeclareMathOperator*{\cone}{cone}$

$P = \left\{ x \middle| Ax \leq b \right\}$

$P = \conv( V ) + \cone( W )$

In [2]:
import numpy as np

## **Incremental algorithm**
In what it follows we implement the double description method

In [3]:
def double_description_method( V, k ) :
  [ m, n ] = V.shape
  I = []
  Jp = []
  Jn = []
  for l in range( 0, n ) :
    if V[k,l] == 0 :
      I.append( l )
    elif V[k,l] > 0 :
      Jp.append( l )
    elif V[k,l] < 0 :
      Jn.append( l )
      
  p = len( I )
  print( I )
  print( Jp )
  print( Jn )
  if p > 0 :
    W = V[:,I]

  if len( Jp ) > 0 and len( Jn ) == 0  :
    W = np.insert( W, np.repeat( p, len( Jp ) ), V[:,Jp], axis = 1 )
    W[k,:] = 0
    
  elif len( Jp ) == 0 and len( Jn ) > 0  :
    W = np.insert( W, np.repeat( p, len( Jn ) ), V[:,Jn], axis = 1 )
    W[k,:] = 0
    
  elif len( Jp ) > 0 and len( Jn ) > 0 :
    
    for i in Jp :
      for j in Jn :
        vki = V[k,i]
        vkj = V[k,j]
#         d = vki - vkj
        d = 1
        if d != 0 :
          w = ( vki / d ) * V[:,j] - ( vkj / d ) * V[:,i]
          if p > 0 :
            W = np.insert( W, p, w, axis = 1 )
            p = W.shape[1]
          else :
            W = w.T
            W.shape = [ m, 1 ]
            p = 1

  return W

In [7]:
d = 2
A = np.ones( [1,d] )
A = np.insert( A, 1, -np.eye( d ), axis = 0 )
print( A )
b = np.zeros( A.shape[0] )
b[0] = 1
print( b )

[[ 1.  1.]
 [-1. -0.]
 [-0. -1.]]
[1. 0. 0.]


In [8]:
[ m, d ] = A.shape
W = np.insert( np.eye( d ), d, A, axis = 0 )
V = np.insert( W, np.repeat( d, d ), -W, axis = 1 )
W = np.insert( np.zeros( [ d, m ] ), d, np.eye( m ), axis = 0 )
V = np.insert( V, np.repeat( 2*d, m ), W, axis = 1 )
del W
print( V )


Vk = np.copy( V )
[ M, N ] = Vk.shape
for j in range( 0, N ) :
  Vk[range( d, M ),j] = Vk[range( d, M ),j] - b

[[ 1.  0. -1. -0.  0.  0.  0.]
 [ 0.  1. -0. -1.  0.  0.  0.]
 [ 1.  1. -1. -1.  1.  0.  0.]
 [-1. -0.  1.  0.  0.  1.  0.]
 [-0. -1.  0.  1.  0.  0.  1.]]


In [9]:
print( Vk )
for k in range( 0, m ) :
  Vk = double_description_method( Vk, d + k )
  print( d + k, ':\n', Vk )

[[ 1.  0. -1. -0.  0.  0.  0.]
 [ 0.  1. -0. -1.  0.  0.  0.]
 [ 0.  0. -2. -2.  0. -1. -1.]
 [-1. -0.  1.  0.  0.  1.  0.]
 [-0. -1.  0.  1.  0.  0.  1.]]
[0, 1, 4]
[]
[2, 3, 5, 6]
2 :
 [[ 1.  0.  0. -1. -0.  0.  0.]
 [ 0.  1.  0. -0. -1.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [-1. -0.  0.  1.  0.  1.  0.]
 [-0. -1.  0.  0.  1.  0.  1.]]
[1, 2, 4, 6]
[3, 5]
[0]
3 :
 [[ 0.  0. -0.  0.  0.  1.]
 [ 1.  0. -1.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.]
 [-0.  0.  0.  0.  0.  0.]
 [-1.  0.  1.  1.  0.  0.]]
[1, 4, 5]
[2, 3]
[0]
4 :
 [[0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]


In [154]:
def double_description_method( A ) :
  [ m, d ] = A.shape
  
  R = np.zeros( [ d, 1 ] )
  for j in range( 0, d ) :
    if A[0,j] > 0 :
      R[j,0] = 1
    elif A[0,j] < 0 :
      R[j,0] = -1
    
  K = list( range( 1, m ) )
  while K != [] :
    I0 = []
    Ip = []
    In = []
    
    i = K.pop( 0 )
    n = R.shape[1]
    u = np.zeros( n )
    for j in range( 0, n ) :
      u[j] = A[i,:].dot( R[:,j] )
      if u[j] > 0 :
        Ip.append( j )
      elif u[j] < 0 : 
        In.append( j )
      else :
        I0.append( j )
    
    if len( Ip ) == 0 and len( In ) > 0 :
      R = np.insert( R, n, -R[:,In + I0], axis = 1 )
      n = R.shape[1]
    elif len( Ip ) > 0 and len( In ) == 0 :
      R = R[:,Ip + I0]
      n = R.shape[1]
    elif len( Ip ) > 0 and len( In ) > 0 :
      S = R[:,Ip + I0]
      n = S.shape[1]
      
      for k in Ip :
        for l in In :
          S = np.insert( S, n, u[k] * R[:,l] - u[l] * R[:,k], axis = 1 )
          n = S.shape[1]
          
      R = np.copy( S )
  
  return R

In [155]:
m = 10
d = 10
# A = np.ones( [1,d] )
# A = -np.insert( A, 1, np.eye( d ), axis = 0 )
A = np.zeros( [m,d] )
for i in range( 0, m ) :
  for j in range( 0, d ) :
    b = np.random.binomial( 2, 0.4, 1 )
    if b == 0 :
      A[i,j] = 0
    elif b == 1 :
      A[i,j] = -1
    else :
      A[i,j] = 1
print( A )

[[-1.  1. -1. -1.  0. -1. -1.  1.  1.  0.]
 [ 0. -1.  0. -1.  1. -1. -1. -1.  1.  0.]
 [ 0. -1. -1. -1.  1.  1. -1. -1.  1. -1.]
 [ 0. -1.  0.  0. -1. -1.  0.  1. -1.  0.]
 [-1.  0. -1. -1. -1. -1. -1. -1. -1. -1.]
 [ 0. -1. -1.  1. -1. -1. -1.  0.  1.  0.]
 [ 0.  1.  0.  0.  0.  0.  0. -1. -1.  0.]
 [-1.  1. -1. -1.  0.  0.  0.  1. -1.  1.]
 [ 1. -1.  0. -1.  0.  0. -1. -1.  1.  0.]
 [ 0.  0. -1. -1. -1. -1. -1.  1.  0. -1.]]
